In [10]:
import polars as pl
from pathlib import Path


#Esto para el script de py
#DATA_PATH = Path(__file__).resolve().parents[1] / "data" / "raw" / "emails.csv"
DATA_PATH = Path.cwd().parent / "data" / "raw" / "emails.csv"

print(DATA_PATH)
#Read csv
emails = pl.read_csv(DATA_PATH)
emails.columns

c:\Users\Martín\Desktop\Proyectos Personales\email-ticket-classifier\data\raw\emails.csv


['id_email', 'user_email', 'subject', 'body', 'date', 'language']